#🎮Trabajo Final -  Consultas Inteligentes sobre Equipos del FIFA 23
Este trabajo final integra técnicas de procesamiento de lenguaje natural con modelos de embeddings y un LLM para permitir consultas en lenguaje natural sobre datos de equipos del **FIFA 23**.

🧾 Los datos fueron obtenidos de un archivo CSV con estadísticas detalladas de clubes como:
- Nombre del equipo
- Liga y país
- Estadísticas ofensivas y defensivas (ataque, mediocampo, defensa)

⚙️ La app utiliza:
- **HuggingFace Embeddings** para vectorizar los datos (GPU si está disponible)
- **Gemini (Google)** como modelo de lenguaje para responder consultas
- **LlamaIndex** para realizar búsquedas semánticas sobre los datos
- **Streamlit** para la interfaz interactiva

##Instalacion de dependencias

In [ ]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-langchain langchain-community langchain-google-genai google-generativeai pandas torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "" #PONER API KEY DE GOOGLE

##Lectura del dataset

In [ ]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')
# ruta_csv = "/content/drive/MyDrive/Trabajo Final Inteligentes - Juan/equipos_filtrados.csv"

df = pd.read_csv("equipos_filtrados.csv")
df.head()

,team_id,team_url,fifa_version,fifa_update,fifa_update_date,team_name,league_id,league_name,league_level,nationality_id,...,off_corners,off_free_kicks,build_up_play_speed,build_up_play_dribbling,build_up_play_passing,build_up_play_positioning,chance_creation_passing,chance_creation_crossing,chance_creation_shooting,chance_creation_positioning
0,10,/team/10/manchester-city/230009,23,9,2023-01-13,Manchester City,13,Premier League,1.0,14,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73,/team/73/paris-saint-germain/230009,23,9,2023-01-13,Paris Saint Germain,16,Ligue 1,1.0,18,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,243,/team/243/real-madrid/230009,23,9,2023-01-13,Real Madrid,53,La Liga,1.0,45,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,/team/5/chelsea/230009,23,9,2023-01-13,Chelsea,13,Premier League,1.0,14,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,/team/9/liverpool/230009,23,9,2023-01-13,Liverpool,13,Premier League,1.0,14,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Transformacion a documentos de texto

In [ ]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_google_genai import ChatGoogleGenerativeAI

# Convertir cada fila a un documento de texto para indexar
documents = []
for _, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    documents.append(Document(text=text))

##Preparacion del modelo de embeddings

In [ ]:
import torch
# Usar embeddings con GPU si hay disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",#all-mpnet-base-v2/all-MiniLM-L12-v2
    device=device
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Configuracion del modelo de lenguaje (LLM)

In [ ]:
from llama_index.core import Settings

# Cliente Gemini como LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Crear contexto de servicio
Settings.llm = llm
Settings.embed_model = embed_model

## Creando indice (no ejecutar si cargamos el index)

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=3)

## Guardar el indice (no ejecutar si cargamos el index)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
persist_dir = "/content/drive/MyDrive/indice_equipos"
index.storage_context.persist(persist_dir=persist_dir)
# Ademas lo guardamos en el content
#index.storage_context.persist(persist_dir="indice_jugadores")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cargar carpeta con indices

In [ ]:
from llama_index.core import load_index_from_storage
from llama_index.core.storage import StorageContext

# Cargar el storage context desde el content
storage_context = StorageContext.from_defaults(persist_dir="indice_equipos")
# Cargar el índice desde el contexto
index = load_index_from_storage(storage_context)
# Crear el query engine normalmente
query_engine = index.as_query_engine(similarity_top_k=3)

Loading llama_index.core.storage.kvstore.simple_kvstore from indice_equipos/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from indice_equipos/index_store.json.


JSONDecodeError: Expecting ',' delimiter: line 1 column 2097153 (char 2097152)

## Consulta usando RAG

In [ ]:
print("🧠 Ejemplos de preguntas que podes hacer:")
print("- ¿Cuanto ataque tiene Real Madrid?\n- ¿Cuanto mediocampo tiene FC Barcelona?\n- ¿En que liga juega Boca Juniors?\n")

while True:
    consulta = input("\n🔎 Ingresá tu consulta (o escribí 'salir' para terminar):\n> ")

    if consulta.lower() == "salir":
        print("👋 Hasta luego.")
        break

    consulta_modificada = consulta.strip() + ". Responde en español."

    respuesta_rag = query_engine.query(consulta_modificada)
    response = llm.invoke(consulta_modificada)
    print("💬 Sin RAG:\n", response.content)
    print("\n✅ Respuesta con RAG:")
    print(str(respuesta_rag))

🧠 Ejemplos de preguntas que podes hacer:
- ¿Cuanto ataque tiene Real Madrid?
- ¿Cuanto mediocampo tiene FC Barcelona?
- ¿En que liga juega Boca Juniors?
- ¿Cuanta defensa tiene Inter Miami CF?

🔎 Ingresá tu consulta (o escribí 'salir' para terminar):
> ¿Cuanta defensa tiene Inter Miami CF?
💬 Sin RAG:
 La defensa del Inter Miami CF es difícil de cuantificar con un número único. No existe una estadística de "defensa total". Sin embargo, podemos evaluar la fortaleza de su defensa considerando varios factores:

*   **Goles encajados:** Esta es una métrica clave. Cuantos menos goles encaje un equipo, generalmente mejor es su defensa. Puedes buscar estadísticas de goles encajados en la temporada actual o en partidos recientes.
*   **Jugadores clave:** Identificar a los defensores centrales, laterales y portero principales del equipo. Su calidad individual y su capacidad para trabajar juntos son cruciales.
*   **Organización táctica:** Observar cómo el equipo se organiza defensivamente, inclu

KeyboardInterrupt: Interrupted by user